In [32]:
# type:ignore
import os
import time
import pyodbc
import pandas as pd

In [33]:
# 建立连接并列出表
def get_tables(conn_str):
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    try:
        # 获取所有表名
        tables = cursor.tables(tableType='TABLE')
        tables_name = [table.table_name for table in tables]
        return tables_name
    finally:
        cursor.close()
        conn.close()

# 读取表数据，返回一个pandas的DataFrame
def get_table_data(conn_str,table_name):
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    try:
        cursor.execute(f"SELECT * FROM {table_name}")
        # 获取列名
        columns = [column[0] for column in cursor.description]
        # 获取数据并创建DataFrame,指定列名
        data = pd.DataFrame.from_records(cursor.fetchall(), columns=columns)
        return data
    finally:
        cursor.close()
        conn.close()
# 查看DF的某列是否有重复值
def check_duplicate(df,column_name):
    return df[column_name].duplicated().sum()

# 去除DF中某列的重复值(只保留第一个)
def remove_duplicate(df,column_name):
    return df.drop_duplicates(subset=[column_name],keep='first')


In [34]:
# 指定MDB文件路径 - 需要包含文件扩展名
mdb_file = r"F:\cache_data\MDB\520181_清镇市.mdb"  # 添加.mdb扩展名

# 检查文件是否存在
if not os.path.exists(mdb_file):
    raise FileNotFoundError(f"找不到MDB文件: {mdb_file}")

# 使用ODBC连接字符串
conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=' + mdb_file + ';'
)

In [35]:
# 获取所有表名
tables = get_tables(conn_str)
print(tables,len(tables))


['CYXX', 'DCYD', 'FJWJ', 'JCFF', 'LDTJDCXX', 'PMXTXDCFCXX', 'PMXTXDCJBXX', 'SFYL', 'SXDM', 'SYS', 'TRHJXZ', 'TRHXXZ', 'TRRZ', 'TRWLXZ', 'YDSQ', 'YPLZ', 'YPLZQD', 'YPZB', 'ZKYP'] 19


In [36]:
# 读取表数据 调查样点属性表  use_col YDBH YDLB CYLX BSJD BSWD JD WD DWJD WDWD DWGC TL YL TS TZ SFCJSWXDTJTYP SFJCJXZC SFTTCPYD TTCPMC
dcyd_df = get_table_data(conn_str,'DCYD')
# 读取指定列
dcyd_df = dcyd_df[['YDBH','YDLB','CYLX','BSJD','BSWD','JD','WD','DWJD','DWWD','DWGC',
                   'TL','YL','TS','TZ','SFCJSWXDTJTYP','SFJCJXZC','SFTTCPYD','TTCPMC']]
print(dcyd_df.shape[0])
# 查看唯一的YDBH
print(dcyd_df['YDBH'].nunique())
# 更改列名TS为TSS避免与其他表格冲突
dcyd_df.rename(columns={'TS':'TSS'},inplace=True)
dcyd_df.sample(1)

1194
1194


,YDBH,YDLB,CYLX,BSJD,BSWD,JD,WD,DWJD,DWWD,DWGC,TL,YL,TSS,TZ,SFCJSWXDTJTYP,SFJCJXZC,SFTTCPYD,TTCPMC
622,5201810103000754,0,1,106.34576352520138000000,26.77568222214493000000,106.34576352520138000000,26.77568222214493000000,106.34515549000000000000,26.77567314000000000000,1183.09558105468750000000,黄壤,典型黄壤,泥质黄壤,中层泥质黄壤,0,0,0,None


In [37]:
# 读取表数据 采样信息属性表 use_col YDBH YPBH YPLX CH
cyxx_df = get_table_data(conn_str,'CYXX')
# 读取指定列
cyxx_df = cyxx_df[['YDBH','YPBH','YPLX','CH']]
print(cyxx_df.shape[0])
# 查看唯一的YDBH 
print(cyxx_df['YDBH'].nunique())
cyxx_df.sample(1)

3239
1194


,YDBH,YPBH,YPLX,CH
500,5201810301100034,520181030110003412,2,2


In [38]:
# 读取表数据 土壤物理性状属性表 use_col YDBH YPBH YYPBH JXZCXSL JXZC1 JXZC2 JXZC3 JXZC4 TRZD SWXDTJT1 SWXDTJT2 SWXDTJT3 SWXDTJT4 SWXDTJT5 SWXDTJT6 SWXDTJT7 
wlxz_df = get_table_data(conn_str,'TRWLXZ')
# 读取指定列
wlxz_df = wlxz_df[['YDBH','YPBH','YYPBH','JXZCXSL','JXZC1','JXZC2','JXZC3','JXZC4','TRZD','SWXDTJT1','SWXDTJT2','SWXDTJT3','SWXDTJT4','SWXDTJT5','SWXDTJT6','SWXDTJT7']]
print(wlxz_df.shape[0])
# 查看唯一的YDBH 
print(wlxz_df['YDBH'].nunique())
wlxz_df.sample(1)


1803
1194


,YDBH,YPBH,YYPBH,JXZCXSL,JXZC1,JXZC2,JXZC3,JXZC4,TRZD,SWXDTJT1,SWXDTJT2,SWXDTJT3,SWXDTJT4,SWXDTJT5,SWXDTJT6,SWXDTJT7
1784,5201810103100077,240423178068b30,520181010310007716,1.1,5.6,23.6,29.6,41.1,壤质黏土,None,None,None,None,None,None,None


In [39]:
# 读取表数据 土壤化学性状属性表 use_col YDBH YPBH YYPBH FGSYHSL PH JHXSZL ECH ECAL SJXZSD CEC JHXYJZL ECA EMG ENA EK SRXYZL DDL SRXNLZ SRXJLZ SRXGLZ
# SRXMLZ SRXTSG SRXTSQG SRXLSG SRXLG LZZL OM TN TP TK TS TB TSI TSE TFE TMN TCU TZN TMO TAL TCA TMG AP SK AK AS1 ASI AFE AMN ACU AZN AB AMO CACO3 FE2O3
hxxz_df = get_table_data(conn_str,'TRHXXZ')
# 读取指定列
hxxz_df = hxxz_df[['YDBH','YPBH','YYPBH','FGSYHSL','PH','JHXSZL','ECH','ECAL','SJXZSD','CEC','JHXYJZL','ECA','EMG','ENA',
                   'EK','SRXYZL','DDL','SRXNLZ','SRXJLZ','SRXGLZ','SRXMLZ','SRXTSG','SRXTSQG','SRXLSG','SRXLG','LZZL',
                   'OM','TN','TP','TK','TS','TB','TSI','TSE','TFE','TMN','TCU','TZN','TMO','TAL','TCA','TMG','AP',
                   'SK','AK','AS1','ASI','AFE','AMN','ACU','AZN','AB','AMO','CACO3','FE2O3']]
print(hxxz_df.shape[0])
# 查看唯一的YDBH 
print(hxxz_df['YDBH'].nunique())
hxxz_df.sample(1)

1612
1194


,YDBH,YPBH,YYPBH,FGSYHSL,PH,JHXSZL,ECH,ECAL,SJXZSD,CEC,...,AS1,ASI,AFE,AMN,ACU,AZN,AB,AMO,CACO3,FE2O3
158,5201810305100058,240423177822b03,520181030510005814,3.4,6.39,/,/,/,/,7.36,...,/,/,/,/,/,/,/,/,/,25.9


In [40]:
# 读取表数据 土壤环境性状属性表 use_col YDBH YPBH YYPBH HG AS2 PB CD CR NI
hjxz_df = get_table_data(conn_str,'TRHJXZ')
# 读取指定列
hjxz_df = hjxz_df[['YDBH','YPBH','YYPBH','HG','AS2','PB','CD','CR','NI']]
print(hjxz_df.shape[0])
# 查看唯一的YDBH 
print(hjxz_df['YDBH'].nunique())
hjxz_df.sample(1)


1612
1194


,YDBH,YPBH,YYPBH,HG,AS2,PB,CD,CR,NI
292,5201810103100082,240423177654b38,520181010310008212,0.149,5.72,19.6,0.20,215,46.3


In [41]:
# 读取表数据 土壤容重属性表 use_col YDBH YPBH TRRZ1 TRRZ2 TRRZ3 TRRZ4 TRRZPJZ 
trrz_df = get_table_data(conn_str, 'TRRZ')
# 读取指定列
trrz_df = trrz_df[['YDBH','YPBH','TRRZ1','TRRZ2','TRRZ3','TRRZ4','TRRZPJZ']]
print(trrz_df.shape[0])
# 查看唯一的YDBH 
print(trrz_df['YDBH'].nunique())
trrz_df.sample(1)


1475
1155


,YDBH,YPBH,TRRZ1,TRRZ2,TRRZ3,TRRZ4,TRRZPJZ
681,5201810103000571,520181010300057120,1.42000000000000000000,1.40000000000000000000,1.40000000000000000000,None,1.41000000000000000000


In [42]:
# 读取表数据 立地条件调查表  use_col YDBH MY MYQT MZ MZQT TDLYLX GZCHD
ldtj_df = get_table_data(conn_str,'LDTJDCXX')
# 读取指定列
ldtj_df = ldtj_df[['YDBH','MY','MYQT','MZ','MZQT','TDLYLX','GZCHD']]
print(ldtj_df.shape[0])
# 查看唯一的YDBH 
print(ldtj_df['YDBH'].nunique())
ldtj_df.sample(1)

1194
1194


,YDBH,MY,MYQT,MZ,MZQT,TDLYLX,GZCHD
52,5201810103000715,"17,18",None,LG,None,0103,12.00000000000000000000


In [43]:
# 读取表数据 剖面形态学调查基本信息调查表  use_col YDBH FSCS YXTCHD FSXTL FSXYL TSXTS TSXTZ 
pmxx_df = get_table_data(conn_str,'PMXTXDCJBXX')
# 读取指定列
pmxx_df = pmxx_df[['YDBH','FSCS','YXTCHD','FSXTL','FSXYL','FSXTS','FSXTZ']]
print(pmxx_df.shape[0])
# 查看唯一的YDBH 
print(pmxx_df['YDBH'].nunique())
pmxx_df.sample(1)

103
103


,YDBH,FSCS,YXTCHD,FSXTL,FSXYL,FSXTS,FSXTZ
36,5201810101100089,6,120,水稻土,脱潜水稻土,黄砂泥田,黄黄砂泥田


In [44]:
# 土壤容重结果 ，基于YDBH
trrz_result = pd.merge(dcyd_df,trrz_df,on='YDBH',how='inner')
print(trrz_result.shape[0])
trrz_result.sample(1)

1475


,YDBH,YDLB,CYLX,BSJD,BSWD,JD,WD,DWJD,DWWD,DWGC,...,SFCJSWXDTJTYP,SFJCJXZC,SFTTCPYD,TTCPMC,YPBH,TRRZ1,TRRZ2,TRRZ3,TRRZ4,TRRZPJZ
249,5201810101100041,1,1,106.33047299999922000000,26.70416499924139200000,106.33047299999922000000,26.70416499924139200000,106.33115755000000000000,26.70464765000000000000,1168.48156738281250000000,...,1,1,0,None,520181010110004123,1.48000000000000000000,1.41000000000000000000,1.45000000000000000000,None,1.45000000000000000000


In [45]:
# 土壤物理性状结果 ，需要连接DCYD和CYXX，基于YDBH
temp_result = pd.merge(dcyd_df,cyxx_df,on='YDBH',how='inner')
print(temp_result.shape[0])
wlxz_result = pd.merge(temp_result,wlxz_df,left_on='YPBH',right_on='YYPBH',how='inner')
print(wlxz_result.shape[0])
wlxz_result.sample(1)

3239
1803


,YDBH_x,YDLB,CYLX,BSJD,BSWD,JD,WD,DWJD,DWWD,DWGC,...,JXZC3,JXZC4,TRZD,SWXDTJT1,SWXDTJT2,SWXDTJT3,SWXDTJT4,SWXDTJT5,SWXDTJT6,SWXDTJT7
159,5201810101000787,0,1,106.44952733102822000000,26.51196966601693500000,106.44952733102822000000,26.51196966601693500000,106.44978454000000000000,26.51188860000000000000,1225.61523437500000000000,...,8.4,39.1,壤质黏土,None,None,None,None,None,None,None


In [46]:
# 土壤化学性状结果 ，需要连接DCYD和CYXX，基于YDBH
temp_result = pd.merge(dcyd_df,cyxx_df,on='YDBH',how='inner')
print(temp_result.shape[0])
hxxz_result = pd.merge(temp_result,hxxz_df,left_on='YPBH',right_on='YYPBH',how='inner')
print(hxxz_result.shape[0])
hxxz_result.sample(1)


3239
1612


,YDBH_x,YDLB,CYLX,BSJD,BSWD,JD,WD,DWJD,DWWD,DWGC,...,AS1,ASI,AFE,AMN,ACU,AZN,AB,AMO,CACO3,FE2O3
114,5201810101000685,0,1,106.40533228096808000000,26.56567980441408500000,106.40533228096808000000,26.56567980441408500000,106.40539974000000000000,26.56505304000000000000,1239.19995117187500000000,...,57.4,333,104,46.0,5.25,2.45,1.12,1.19,/,/


In [47]:
# 土壤环境性状结果 ，需要连接DCYD和CYXX，基于YDBH
temp_result = pd.merge(dcyd_df,cyxx_df,on='YDBH',how='inner')
print(temp_result.shape[0])
hjxz_result = pd.merge(temp_result,hjxz_df,left_on='YPBH',right_on='YYPBH',how='inner')
print(hjxz_result.shape[0])
hjxz_result.sample(1)


3239
1612


,YDBH_x,YDLB,CYLX,BSJD,BSWD,JD,WD,DWJD,DWWD,DWGC,...,CH,YDBH_y,YPBH_y,YYPBH,HG,AS2,PB,CD,CR,NI
523,5201810103000161,0,1,106.33112375303112000000,26.68948787189362200000,106.33112375303112000000,26.68948787189362200000,106.33165069000000000000,26.68948370000000000000,1225.09606933593750000000,...,None,5201810103000161,231127077663b34,520181010300016110,0.487,70.3,75.9,0.41,142,72.8


In [48]:
def process_merge_result(df):
    """
    处理合并后的DataFrame,去除重复列并重命名列名
    Args:
        df: 需要处理的DataFrame
    Returns:
        处理后的DataFrame
    """
    # 去除重复列
    result = df.loc[:, ~df.columns.str.endswith('_y')]
    # 创建副本避免SettingWithCopyWarning
    result = result.copy()
    # 重命名列,将_x替换为空
    result = result.rename(columns=lambda x: x.replace('_x', ''))
    return result

# 处理各个结果
result_wlxz = process_merge_result(wlxz_result)
result_hxxz = process_merge_result(hxxz_result)
result_hjxz = process_merge_result(hjxz_result) 
result_trrz = process_merge_result(trrz_result)


In [49]:
# 过滤数据
# 过滤数据 土壤容重结果 TRRZPJZ不为空
filter_trrz_result = result_trrz[
    (~result_trrz['TRRZPJZ'].isna()) & # 过滤 NaN
    (result_trrz['TRRZPJZ'] != 'None') & # 过滤字符串 'None' 
    (result_trrz['TRRZPJZ'].notna()) & # 再次确认过滤 NaN
    (result_trrz['TRRZPJZ'] != '') # 过滤空字符串

]
print(filter_trrz_result.shape[0])
filter_trrz_result.sample(1)
# 删除YYPH重复项
filter_trrz_result = remove_duplicate(filter_trrz_result,'YPBH')
print(filter_trrz_result.shape[0])
filter_trrz_result.sample(1)
# 删除YDBH重复项
filter_trrz_result = remove_duplicate(filter_trrz_result,'YDBH')
print(filter_trrz_result.shape[0])
filter_trrz_result.sample(1)
# 查看处理前后的YDBH数量
print(f"原YDBH数量：{trrz_df['YDBH'].nunique()}")
print(f"去重后YDBH数量：{filter_trrz_result['YDBH'].nunique()}")


1475
1475
1155
原YDBH数量：1155
去重后YDBH数量：1155


In [50]:
# 过滤数据 土壤物理性状结果 YPLX为1或2，CH为1或空值
# filter_wlxz_result = result_wlxz[
#     (result_wlxz['YPLX'].isin(['1','2',])) & # YPLX为1或2
#     ((result_wlxz['CH'].isna()) | (result_wlxz['CH'] == '1')) # CH为空值或1
# ]
filter_wlxz_result = result_wlxz[
    ((result_wlxz['CH'].isna()) | (result_wlxz['CH'] == '1')) # CH为空值或1
]
print(filter_wlxz_result.shape[0])
filter_wlxz_result.sample(1)
# 删除YYPBH
filter_wlxz_result = remove_duplicate(filter_wlxz_result,'YYPBH')
print(filter_wlxz_result.shape[0])
filter_wlxz_result.sample(1)
# 删除YDBH
filter_wlxz_result = remove_duplicate(filter_wlxz_result,'YDBH')
print(filter_wlxz_result.shape[0])
filter_wlxz_result.sample(1)
# 查看处理前后的YDBH数量
print(f"原YDBH数量：{wlxz_df['YDBH'].nunique()}")
print(f"去重后YDBH数量：{filter_wlxz_result['YDBH'].nunique()}")



1423
1382
1194
原YDBH数量：1194
去重后YDBH数量：1194


In [51]:
# 过滤数据 土壤化学性状结果 
filter_hxxz_result = result_hxxz[
    ((result_hxxz['CH'].isna()) | (result_hxxz['CH'] == '1')) # CH为空值或1
]
print(filter_hxxz_result.shape[0])
filter_hxxz_result.sample(1)
# 删除YYPBH
filter_hxxz_result = remove_duplicate(filter_hxxz_result,'YYPBH')
print(filter_hxxz_result.shape[0])
filter_hxxz_result.sample(1)
# 删除YDBH
filter_hxxz_result = remove_duplicate(filter_hxxz_result,'YDBH')
print(filter_hxxz_result.shape[0])
filter_hxxz_result.sample(1)
# 查看处理前后的YDBH数量
print(f"原YDBH数量：{hxxz_df['YDBH'].nunique()}")
print(f"去重后YDBH数量：{filter_hxxz_result['YDBH'].nunique()}")

1232
1194
1194
原YDBH数量：1194
去重后YDBH数量：1194


In [52]:
# 过滤数据 土壤环境性状结果 
filter_hjxz_result = result_hjxz[
    ((result_hjxz['CH'].isna()) | (result_hjxz['CH'] == '1')) # CH为空值或1
]
print(filter_hjxz_result.shape[0])
filter_hjxz_result.sample(1)
# 删除YYPBH
filter_hjxz_result = remove_duplicate(filter_hjxz_result,'YYPBH')
print(filter_hjxz_result.shape[0])
filter_hjxz_result.sample(1)
# 删除YDBH
filter_hjxz_result = remove_duplicate(filter_hjxz_result,'YDBH')
print(filter_hjxz_result.shape[0])
filter_hjxz_result.sample(1)
# 查看处理前后的YDBH数量
print(f"原YDBH数量：{hjxz_df['YDBH'].nunique()}")
print(f"去重后YDBH数量：{filter_hjxz_result['YDBH'].nunique()}")

1232
1194
1194
原YDBH数量：1194
去重后YDBH数量：1194


In [53]:
# 处理后检查机械组成（filter_wlxz_result中TRZD列不为空且不为'/'的数据）和原表数据涉及的YDBH是否一致
jxzc_df = filter_wlxz_result[
    (filter_wlxz_result['TRZD'].notna()) & # 过滤NaN
    (filter_wlxz_result['TRZD'] != '/') # 过滤'/'
]
# 处理前机械组成YDBH数量
jxzc_df_before = wlxz_df[
    (wlxz_df['TRZD'].notna()) & # 过滤NaN
    (wlxz_df['TRZD'] != '/') # 过滤'/'
]
# 查看处理前后的YDBH数量及是否检测(SFJCJXZC列值为'1')的数量
print(f"处理前机械组成YDBH数量：{jxzc_df_before['YDBH'].nunique()}")
print(f"处理后机械组成YDBH数量：{jxzc_df['YDBH'].nunique()}")
print(f"是否检测机械组成YDBH数量：{jxzc_df[jxzc_df['SFJCJXZC'] == '1']['YDBH'].nunique()}")


处理前机械组成YDBH数量：649
处理后机械组成YDBH数量：584
是否检测机械组成YDBH数量：584


In [54]:
# 处理后水团组成（filter_wlxz_result中SWXDTJT7列不为空且不为'/'的数据）和原表数据涉及的YDBH是否一致
stzc_df = filter_wlxz_result[
    (filter_wlxz_result['SWXDTJT7'].notna()) & # 过滤NaN
    (filter_wlxz_result['SWXDTJT7'] != '/') # 过滤'/'
]
# 处理前机械组成YDBH数量
stzc_df_before = wlxz_df[
    (wlxz_df['SWXDTJT7'].notna()) & # 过滤NaN
    (wlxz_df['SWXDTJT7'] != '/') # 过滤'/'
]
# 查看处理前后的YDBH数量及是否采集水团样品(SFCJSWXDTJTYP列值为'1')
print(f"处理前水团组成YDBH数量：{stzc_df_before['YDBH'].nunique()}")
print(f"处理后水团组成YDBH数量：{stzc_df['YDBH'].nunique()}")
print(f"是否采集水团样品YDBH数量：{stzc_df[stzc_df['SFCJSWXDTJTYP'] == '1']['YDBH'].nunique()}")


处理前水团组成YDBH数量：188
处理后水团组成YDBH数量：90
是否采集水团样品YDBH数量：90


In [55]:
# 连接所有表 filter_wlxz_result filter_hxxz_result filter_hjxz_result filter_trrz_result,ldtj_df pmxx_df 基于YDBH
result_all = pd.merge(filter_wlxz_result,filter_hxxz_result,on='YDBH',how='left')
result_all = process_merge_result(result_all)
result_all = pd.merge(result_all,filter_hjxz_result,on='YDBH',how='left')
result_all = process_merge_result(result_all)
result_all = pd.merge(result_all,filter_trrz_result,on='YDBH',how='left')
result_all = process_merge_result(result_all)
result_all = pd.merge(result_all,ldtj_df,on='YDBH',how='left')
result_all = process_merge_result(result_all)
result_all = pd.merge(result_all,pmxx_df,on='YDBH',how='left')
result_all = process_merge_result(result_all)

In [56]:
result_all

,YDBH,YDLB,CYLX,BSJD,BSWD,JD,WD,DWJD,DWWD,DWGC,...,MZ,MZQT,TDLYLX,GZCHD,FSCS,YXTCHD,FSXTL,FSXYL,FSXTS,FSXTZ
0,5201810101000001,0,1,106.44840860965174000000,26.49391778773308200000,106.44840860965174000000,26.49391778773308200000,106.44855652000000000000,26.49410836000000000000,1217.25952148437500000000,...,LG,None,0101,18.00000000000000000000,NaN,NaN,NaN,NaN,NaN,NaN
1,5201810101000017,0,1,106.35354407164344000000,26.48385506755995000000,106.35354407164344000000,26.48385506755995000000,106.35362189000000000000,26.48365773000000000000,1224.90502929687500000000,...,LG,None,0101,16.00000000000000000000,NaN,NaN,NaN,NaN,NaN,NaN
2,5201810101000028,0,1,106.38075429098606000000,26.72977954211535000000,106.38075429098606000000,26.72977954211535000000,106.38049880000000000000,26.73004027000000000000,1294.05700683593750000000,...,LG,None,0101,16.00000000000000000000,NaN,NaN,NaN,NaN,NaN,NaN
3,5201810101000032,0,1,106.34309171864003000000,26.60446880143308200000,106.34309171864003000000,26.60446880143308200000,106.34286229000000000000,26.60470857000000000000,1260.32397460937500000000,...,LG,None,0101,13.00000000000000000000,NaN,NaN,NaN,NaN,NaN,NaN
4,5201810101000043,0,1,106.25535678622636000000,26.60308942315300000000,106.25535678622636000000,26.60308942315300000000,106.25553551000000000000,26.60319016000000000000,1281.44885253906250000000,...,LG,None,0101,12.00000000000000000000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,5201810404000299,0,1,106.43928386156416000000,26.55744781152668600000,106.43928386156416000000,26.55744781152668600000,106.43936277000000000000,26.55737715000000000000,1245.06457519531250000000,...,LG,None,0404,None,NaN,NaN,NaN,NaN,NaN,NaN
1190,5201810404000362,0,1,106.49823526111665000000,26.50770732131011400000,106.49823526111665000000,26.50770732131011400000,106.49821197000000000000,26.50807989000000000000,1250.65258789062500000000,...,LG,None,0404,None,NaN,NaN,NaN,NaN,NaN,NaN
1191,5201810404000376,0,1,106.43804178912679000000,26.56270085196947000000,106.43804178912679000000,26.56270085196947000000,106.43778034000000000000,26.56275213000000000000,1224.80554199218750000000,...,LG,None,0404,None,NaN,NaN,NaN,NaN,NaN,NaN
1192,5201810404000573,0,1,106.48669159051622000000,26.53284366371029000000,106.48330700000000000000,26.52157800000000000000,106.48327639000000000000,26.52143462000000000000,1262.94897460937500000000,...,LG,None,0404,None,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
# 导出为xlsx,总表
save_path = r"G:\soil_property_result\qzs\table"
os.makedirs(save_path,exist_ok=True)
result_all.to_excel(os.path.join(save_path,f'result_all_{time.strftime("%Y%m%d_%H%M%S")}.xlsx'),index=False)

In [58]:
# 导出为xlsx,用于分析
# use_col YDBH DWJD DWWD DWGC TRZD PH CEC OM TN TP TK TSE AP SK AK HG AS2 PB CD CR TRRZPJZ GZCHD_LDTJ GZCHD YXTCHD
ana_df = result_all[['YDBH','DWJD','DWWD','DWGC','TRZD','PH','CEC','OM','TN','TP','TK','TSE','AP','SK','AK',
                     'HG','AS2','PB','CD','CR','TRRZPJZ','GZCHD','YXTCHD','FSXTL', 'FSXYL', 'FSXTS', 'FSXTZ']]
# 重命名列
ana_df = ana_df.copy()
ana_df.rename(columns={'TRRZPJZ':'TRRZ'},inplace=True)
ana_df.to_excel(os.path.join(save_path,f'result_ana_df_{time.strftime("%Y%m%d_%H%M%S")}.xlsx'),index=False)